# pytest

O `pytest` é uma biblioteca Python para escrever e executar testes de software de maneira simples e eficaz. Ele oferece uma abordagem flexível e poderosa para realizar testes em comparação com o módulo `unittest`, padrão do Python. Sua flexibilidade e recursos adicionais o tornam uma escolha popular para testes automatizados em Python, sendo amplamente adotado pela comunidade. 

**Obs.:** Vale ressaltar que é preciso baixar o `pytest` previamente com o `pip`.

O pytest também oferece um recurso poderoso chamado *fixtures*, que permite a configuração prévia e limpeza após os testes, facilitando a reutilização de código e a manutenção dos testes.

### Command Line Interface (CLI)

Os testes criados com o `pytest` normalmente são executados como interface de linha de comando (CLI) no terminal, ou com a ferramenta *Testing* do VSCode. Em ambos os cenários, a biblioteca identificará todos os arquivos de teste que tenham nomes prefixados com "test_", executando-os em ordem. 

In [9]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.3, pluggy-1.3.0
rootdir: /home/brunomarcelino/github/estudos-com-py/deploy/testing
collected 0 items                                                              

============================ no tests ran in 0.00s =============================


É possível passar diversos argumentos de linha de comando para o `pytest`. Eles oferecem flexibilidade para personalizar a execução dos testes de acordo com as necessidades do projeto. Dentre eles, os mais comuns são:

* Execução de Testes:
  * `pytest`: Executa todos os testes na pasta atual e subpastas.
  * `pytest nome_do_arquivo.py`: Executa os testes no arquivo específico.
  * `pytest -k "substring"`: Executa apenas os testes cujos nomes contenham a substring fornecida.
  * `pytest -m "nome_da_marcação"`: Executa os testes marcados com a marcação específica.

* Saída e Relatórios:
  * `pytest -v`: Exibe informações detalhadas (verbosas) sobre os testes.
  * `pytest --tb=line`: Mostra apenas uma linha para falhas (tracebacks).
  * `pytest --junitxml=nome_do_arquivo.xml`: Gera um relatório no formato JUnit XML.
  * `pytest --cov=nome_do_pacote`: Calcula a cobertura de código para o pacote especificado.

* Marcadores e Seleção de Testes:
  * `pytest -m "not marcação"`: Executa todos os testes que não possuem a marcação especificada.
  * `pytest -k "not substring"`: Executa todos os testes cujos nomes não contenham a substring especificada.
  * `pytest --lf`: Executa somente os testes falhos da execução anterior.
  * `pytest --ff`: Executa somente os testes falhos da execução anterior, reexecutando todos os testes após eles.

* Fixtures e Escopos:
  * `pytest --fixtures`: Mostra informações sobre todas as fixtures disponíveis.
  * `pytest --setup-show`: Mostra as informações de setup/teardown durante a execução dos testes.

* Configurações e Plugins:
  * `pytest --version`: Exibe a versão do pytest.
  * `pytest -p nome_do_plugin`: Adiciona plugins específicos durante a execução dos testes.

* Seleção de Classes e Métodos:
  * `pytest nome_do_arquivo.py::NomeDaClasse`: Executa os testes apenas para a classe específica.
  * `pytest nome_do_arquivo.py::NomeDaClasse::nome_do_metodo`: Executa apenas um método específico dentro de uma classe.

### Marks

Os *marks* (marcadores) no `pytest` são utilizados para marcar funções, classes ou módulos de teste com rótulos específicos, permitindo categorizar e identificar testes de forma a controlar sua execução e aplicar ações específicas. Isso ajuda na execução seletiva e na gestão de testes em projetos grandes ou complexos.

Os marcadores geralmente são usados para filtrar a execução de testes, realizar a execução condicional com diferentes configurações ou até mesmo para gerar relatórios específicos (como identificar testes de performance ou integração).

Para criar um marcador em uma função de teste, você pode usar o decorador `@pytest.mark.:nome_do_marcador:`.

In [10]:
import pytest

@pytest.mark.fast
def test_calculo_rapido():
    # código do teste
    pass

@pytest.mark.slow
def test_calculo_lento():
    # código do teste
    pass

Para executar testes com base em marcadores específicos, use o parâmetro `-m` seguido pelo nome do marcador, conforme o exemplo abaixo:

```
  pytest -m fast  # Executa testes marcados como "fast"
  pytest -m "not slow"  # Executa testes que NÃO estão marcados como "slow"
```

Você também pode definir marcadores personalizados em um arquivo `pytest.ini` ou `setup.cfg`. Por exemplo:

```
  # pytest.ini
  [pytest]
  markers =
      slow: testes lentos
      fast: testes rápidos
```



### Fixtures

As fixtures no pytest são funções que ajudam na configuração, fornecendo um **contexto inicial** para os testes e na limpeza após a sua execução. Elas permitem a criação de dados de teste (*mocks*), inicialização de recursos ou configurações específicas para serem usadas em diferentes conjuntos de testes. 

Uma fixture é definida usando o decorador `@pytest.fixture` em uma função. Quando essa função é nomeada nos argumentos de uma função de teste, o pytest a reconhece e a executa antes do teste.

In [6]:
import pytest

# Definindo uma fixture
@pytest.fixture
def setup_data():
  data = {'key': 'value'}
  return data

# Utilizando a fixture em um teste
def test_data(setup_data):
  assert 'key' in setup_data
  print("Teste Finalizado!")

As fixtures podem ter diferentes escopos, como função, classe, módulo ou sessão. O escopo define por quanto tempo a fixture será mantida. Por padrão, o escopo é função, o que significa que a fixture é executada uma vez por teste. Entretanto, é possível especificar escopos maiores para reutilização entre testes.

In [3]:
import pytest

# Fixture com escopo de classe
@pytest.fixture(scope="class")
def setup_data():
    data = {'key': 'value'}
    print("\nSetting up data...")
    return data

# Classe de teste
class TestClassScope:
    # Usando a fixture setup_data nesta classe de teste
    def test_data_exists(self, setup_data):
        assert 'key' in setup_data
        assert setup_data['key'] == 'value'
        print("Teste Finalizado!")

    def test_data_modified(self, setup_data):
        setup_data['new_key'] = 'new_value'
        assert 'new_key' in setup_data
        assert setup_data['new_key'] == 'new_value'
        print("Teste Finalizado!")

    def test_data_unmodified(self, setup_data):
        assert 'new_key' not in setup_data
        print("Teste Finalizado!")

Neste exemplo, a fixture `setup_data` é definida com um escopo de classe (scope="class"). Isso significa que a função `setup_data` será executada uma vez para cada classe de teste, fornecendo o mesmo estado inicial para todos os testes dentro dessa classe.

As funções de teste (`test_data_exists`, `test_data_modified` e `test_data_unmodified`) recebem a fixture `setup_data` como argumento. Como o escopo é de classe, todas essas funções de teste compartilham o mesmo estado inicial definido pela fixture `setup_data`.

Ao executar estes testes, a fixture `setup_data` será executada uma vez para toda a classe de teste, mantendo o mesmo estado inicial para os testes dentro dessa classe.